In [1]:
# !pip install torch

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score, 
                             mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error)

from tqdm.auto import tqdm
import copy

np.random.seed(12345)
torch.manual_seed(123)

# ========== Data ==========

In [3]:
data = pd.read_csv('dataset.csv')

In [4]:
data

,x1,x2,y
0,2,1,9
1,2,-1,1
2,-2,2,6
3,1,2,11
4,-2,3,9
5,2,0,5
6,-1,-1,-2
7,-2,1,3
8,0,0,2
9,1,-1,0


In [5]:
X = data[['x1', 'x2']].to_numpy()

In [6]:
y = data['y'].to_numpy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

# ========== Model ==========

In [8]:
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'

In [9]:
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)

X_val = torch.from_numpy(X_val).float().to(device)
y_val = torch.from_numpy(y_val).float().to(device)

In [10]:
layers = []

layers.append(nn.Linear(X.shape[1], 5))
layers.append(nn.ReLU())

layers.append(nn.Linear(5, 4))
layers.append(nn.ReLU())

layers.append(nn.Linear(4, 1))

model = nn.Sequential(*layers).to(device)

In [11]:
best_mse = np.inf

alphas = np.logspace(-3, 2, 100)

for alpha in tqdm(alphas):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_function = nn.MSELoss()
    for i in tqdm(range(500)):
        model.train()
        optimizer.zero_grad()

        y_pred_train = model(X_train)
        y_pred_train = y_pred_train.reshape(-1)

        l2_norm = 0
        for layer in model.children():
            if isinstance(layer, nn.Linear):
                for params in layer.parameters():
                    l2_norm += sum(p.pow(2.0).sum() for p in params)

        loss = loss_function(y_pred_train, y_train) + alpha*l2_norm
        loss.backward()

        optimizer.step()
        
    with torch.no_grad():
        model.eval()
        y_pred_val = model(X_val)
        
    if np.isnan(y_pred_val.min()):
        continue
        
    mse = mean_squared_error(y_val, y_pred_val)
    
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_alpha = alpha

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [12]:
best_alpha

0.002848035868435802

In [13]:
for name, param in best_model.named_parameters():
    print(name, param)
    print()

0.weight Parameter containing:
tensor([[-0.2939, -0.1412],
        [ 0.1516,  1.0195],
        [ 0.2429,  0.8256],
        [-0.7711, -0.9497],
        [ 0.8180,  0.8359]], requires_grad=True)

0.bias Parameter containing:
tensor([-0.1642,  0.7563, -0.2867,  0.0087, -0.4973], requires_grad=True)

2.weight Parameter containing:
tensor([[-0.0744, -0.1691,  0.3024, -0.0878,  0.2580],
        [ 0.2732,  0.0389,  0.0863,  0.4708, -0.1703],
        [ 0.1682, -0.4057,  0.2156,  0.9883, -0.3125],
        [ 0.3252,  1.1683,  0.3427, -0.0866,  1.1606]], requires_grad=True)

2.bias Parameter containing:
tensor([-0.0197, -0.2510,  0.4560,  0.0616], requires_grad=True)

4.weight Parameter containing:
tensor([[ 0.1692, -0.2429, -1.1130,  1.6146]], requires_grad=True)

4.bias Parameter containing:
tensor([0.6371], requires_grad=True)



# ========== Prediction ==========

In [14]:
X = pd.read_csv('incoming_data.csv').to_numpy()

In [15]:
X = torch.from_numpy(X).float().to(device)

In [16]:
with torch.no_grad():
    best_model.eval()
    y_pred = best_model(X)
    
y_pred

tensor([[12.9991],
        [-2.9426],
        [ 5.0191],
        [-0.9732]])